In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib.pyplot import *
from torch.optim import Adam

In [2]:
#Data Loading
#Load train set and test set and normalize the images in range [-1,1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

#50000 images training
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
#We load 4 samples per batchreduce the traininset to 12500
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

#print(len(trainset))
#10000 images test
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
#CNN definition
class Net(nn.Module):
    def __init__(self): #here you define de frame 
        super(Net, self).__init__()
        #input images: 32x32x3
        self.conv1 = nn.Conv2d(3, 6, 5)
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size) stride=1, padding=0, dilation=1
        self.pool = nn.MaxPool2d(2, 2)
        #nn.MaxPool2d(kernel_size, stride=None)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        
        #FC layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        #applies linear transformation to the incoming data y=x*A'+b
        #torch.nn.Linear(in_features, out_features, bias=True)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): #here you define the connections between the different layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        # keep same infos but change the shape in a 2d tensor
        x = F.relu(self.fc1(x))
        #apply activation fct with RELU
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
#params = list(net.parameters())
#print(len(params))
#print(params[2].size())  

10
torch.Size([16, 6, 5, 5])


In [7]:
#Define Loss function and optimizer
# Loss Function: cross entropy
# Optimizer: SGD
criterion = nn.CrossEntropyLoss() #loss function
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #SGD method to fit weights and biases to groundtruth
#optimizer = Adam(net.parameters())

In [8]:
#Training network over 2 epochs
epochs=8
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print('inputs',inputs.size(),'labels',labels.size(),'outpu',outputs.size())
        #print('outputs:',outputs.type(),'labels:',labels.type())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()#for backpropagation we use SGD here implemented cross entropy loss
        #here we update weights in net
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 1.008
[1,  4000] loss: 1.006
[1,  6000] loss: 1.006
[1,  8000] loss: 1.027
[1, 10000] loss: 1.033
[1, 12000] loss: 1.043
[2,  2000] loss: 0.957
[2,  4000] loss: 0.945
[2,  6000] loss: 0.959
[2,  8000] loss: 0.970
[2, 10000] loss: 0.990
[2, 12000] loss: 0.988
[3,  2000] loss: 0.895
[3,  4000] loss: 0.898
[3,  6000] loss: 0.927
[3,  8000] loss: 0.944
[3, 10000] loss: 0.930
[3, 12000] loss: 0.958
[4,  2000] loss: 0.853
[4,  4000] loss: 0.853
[4,  6000] loss: 0.900
[4,  8000] loss: 0.893
[4, 10000] loss: 0.897
[4, 12000] loss: 0.904
[5,  2000] loss: 0.802
[5,  4000] loss: 0.847
[5,  6000] loss: 0.853
[5,  8000] loss: 0.874
[5, 10000] loss: 0.863
[5, 12000] loss: 0.882
[6,  2000] loss: 0.797
[6,  4000] loss: 0.782
[6,  6000] loss: 0.804
[6,  8000] loss: 0.837
[6, 10000] loss: 0.866
[6, 12000] loss: 0.840
[7,  2000] loss: 0.724
[7,  4000] loss: 0.790
[7,  6000] loss: 0.783
[7,  8000] loss: 0.811
[7, 10000] loss: 0.802
[7, 12000] loss: 0.826
[8,  2000] loss: 0.723
[8,  4000] 

In [9]:
#training saving
PATH = './cifar_net_8epochs.pth'
torch.save(net.state_dict(), PATH)

In [10]:
#Test
#dataiter = iter(testloader)
#images, labels = dataiter.next()

PATH = './cifar_net_4epochs.pth'
#Load previously net from choosen training
net = Net()
net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad(): #desactivate autograd engine ( used to perform validation )
    for data in testloader:
        images, labels = data
        #print('images',images.size(),'labels',labels.size())
        outputs = net(images) #apply our updated CNN to images of test set
        #print ('outputs',outputs.size())
        _, predicted = torch.max(outputs.data, 1) #torch.max(input, dim) return maximum value of all element from input tensor in the given dim
        total += labels.size(0) #count the number of labels with right shape
        correct += (predicted == labels).sum().item() #count the number of right labels 

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))
#0.63 accuracy with SGD + momentum optimizer
#0.58 accury with Adam optimizer

Accuracy of the network on the 10000 test images: 61 %


In [11]:
#Optional but good to know

# print images: Preview of the results of our network ##########
#imshow(torchvision.utils.make_grid(images))

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

net = Net()
net.load_state_dict(torch.load(PATH))

outputs = net(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))
################################################################


#How to know the performance of each class #############
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
############################################################



GroundTruth:    cat   dog   car horse
Predicted:    cat   dog   car horse
Accuracy of plane : 55 %
Accuracy of   car : 79 %
Accuracy of  bird : 49 %
Accuracy of   cat : 41 %
Accuracy of  deer : 51 %
Accuracy of   dog : 47 %
Accuracy of  frog : 72 %
Accuracy of horse : 68 %
Accuracy of  ship : 81 %
Accuracy of truck : 68 %
